In [1]:
import numpy as np
from scipy.io import loadmat
from scipy.optimize import minimize

In [2]:
data=loadmat(r"C:\Users\Shahed\OneDrive - Hashemite University\Classification_with_Multivariables_data-1\Classification with Multivariables data.mat")

In [3]:
data

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Sun Oct 16 13:09:09 2011',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 'y': array([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 9],
        [ 9]], dtype=uint8)}

In [4]:
data['X']

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [5]:
data['X'].shape

(5000, 400)

In [6]:
data['y']

array([[10],
       [10],
       [10],
       ...,
       [ 9],
       [ 9],
       [ 9]], dtype=uint8)

In [7]:
data['y'].shape

(5000, 1)

In [8]:
rows=data['X'].shape[0]
parameters=data['X'].shape[1]

In [9]:
all_theta=np.zeros((10,parameters+1))
all_theta.shape

(10, 401)

In [10]:
x=np.insert(data['X'],0,values=np.ones(rows),axis=1)
x

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]])

In [11]:
x.shape

(5000, 401)

In [12]:
theta=np.zeros(parameters+1)
theta.shape

(401,)

In [13]:
y0=np.array([1 if label==0 else 0 for label in data['y']])
y0

array([0, 0, 0, ..., 0, 0, 0])

In [14]:
y0=np.reshape(y0,(rows,1))
y0

array([[0],
       [0],
       [0],
       ...,
       [0],
       [0],
       [0]])

In [15]:
np.unique(data['y'])

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10], dtype=uint8)

In [16]:
def sigmoid(z):
    return 1 / (1 + np.exp(-z))

In [17]:
def cost(theta,x,y,learning_rate):
    x=np.matrix(x)
    y=np.matrix(y)
    theta=np.matrix(theta)
    first=np.multiply(-y,np.log(sigmoid(x*theta.T)))
    second=np.multiply(1-y,np.log(1-sigmoid(x*theta.T)))
    reg=(learning_rate/2*len(x))*np.sum(np.power(theta[:,1:],2))
    return np.sum(first-second)/len(x)+reg

In [18]:
def gradient(theta,x,y,learning_rate):
    x=np.matrix(x)
    y=np.matrix(y)
    theta=np.matrix(theta)
    
    parameters=int(theta.ravel().shape[1])
    grad=np.zeros(parameters)

    error=sigmoid(x*theta.T)-y

    for i in range(parameters):
        term=np.multiply(error,x[:, i])
        
        if(i==0):
            grad[i]=np.sum(term)/len(x)
        else:    
            grad[i]=(np.sum(term)/len(x))+((learning_rate/len(x))*theta[:,i])

    return grad

In [19]:
def one_vs_all(X,y,num_labels,learning_rate):
    rows=X.shape[0] 
    parameters=X.shape[1] 
    all_theta=np.zeros((num_labels,parameters+1))
    X=np.insert(X,0,values=np.ones(rows),axis=1)
    
    print('all_theta shape ',all_theta.shape)
    print('X shape ',X.shape)
    
    for i in range(1,num_labels+1):
        theta=np.zeros(parameters+1)
        y_i=np.array([1 if label==i else 0 for label in y])
        y_i=np.reshape(y_i,(rows,1))
        
        fmin=minimize(fun=cost,x0=theta,args=(X,y_i,learning_rate),method='TNC',jac=gradient)
        all_theta[i-1,:]=fmin.x
    
    return all_theta

In [20]:
all_theta=one_vs_all(data['X'],data['y'],10,1)
all_theta

all_theta shape  (10, 401)
X shape  (5000, 401)


C:\Users\Shahed\AppData\Local\Temp\ipykernel_19320\2924932602.py:17: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  grad[i]=(np.sum(term)/len(x))+((learning_rate/len(x))*theta[:,i])


array([[-3.70247934e-05,  0.00000000e+00,  0.00000000e+00, ...,
        -2.24803606e-10,  2.31962908e-11,  0.00000000e+00],
       [-8.96250684e-05,  0.00000000e+00,  0.00000000e+00, ...,
         7.26120769e-09, -6.19965249e-10,  0.00000000e+00],
       [-8.39553339e-05,  0.00000000e+00,  0.00000000e+00, ...,
        -7.61695562e-10,  4.64917627e-11,  0.00000000e+00],
       ...,
       [-7.00832561e-05,  0.00000000e+00,  0.00000000e+00, ...,
        -6.92009170e-10,  4.29241575e-11,  0.00000000e+00],
       [-7.65188003e-05,  0.00000000e+00,  0.00000000e+00, ...,
        -8.09503381e-10,  5.31058777e-11,  0.00000000e+00],
       [-6.63412498e-05,  0.00000000e+00,  0.00000000e+00, ...,
        -3.49765933e-09,  1.13668559e-10,  0.00000000e+00]])

In [21]:
def predict_all(x,all_theta):
    rows=x.shape[0]
    parameters=x.shape[1]
    labels_numbers=all_theta.shape[0]
    
    x=np.insert(x, 0, values=np.ones(rows), axis=1)
    x=np.matrix(x)
    all_theta=np.matrix(all_theta)
    
    h=sigmoid(x*all_theta.T)
    h_maxindex=np.argmax(h,axis=1)
    h_maxindex=h_maxindex+1
    
    return h_maxindex

In [22]:
y_predecte=predict_all(data['X'],all_theta)
y_predecte

matrix([[10],
        [10],
        [10],
        ...,
        [ 9],
        [ 7],
        [10]], dtype=int64)

In [23]:
correct=[1 if predecte==y else 0 for (predecte,y) in zip(y_predecte,data['y'])]
accuracy=(sum(correct))/float(len(correct))
print('accuracy = {0}%'.format(accuracy*100))

accuracy = 74.6%
